In [17]:
import json
import pandas as pd
import numpy as np
import os
import re

from sqlalchemy import create_engine

#from config import db_password

import time

In [18]:
data = pd.read_csv (r'Resources\austinHousingData.csv')
Austin_Housing_Market_df = pd.DataFrame(data, columns= ['zpid', 'city', 'streetAddress', 'zipcode', 'latitude', 'longitude', 'latestPrice', 'numPriceChanges', 'latest_saledate', 'latest_salemonth', 'latest_saleyear', 'homeType', 'lotSizeSqFt', 'livingAreaSqFt', 'numOfBedrooms', 'numOfBathrooms', 'propertyTaxRate'])
Austin_Housing_Market_df.head(10)

,zpid,city,streetAddress,zipcode,latitude,longitude,latestPrice,numPriceChanges,latest_saledate,latest_salemonth,latest_saleyear,homeType,lotSizeSqFt,livingAreaSqFt,numOfBedrooms,numOfBathrooms,propertyTaxRate
0,111373431,pflugerville,14424 Lake Victor Dr,78660,30.430632,-97.663078,305000.0,5,2019-09-02,9,2019,Single Family,6011.0,2601.0,4,3.0,1.98
1,120900430,pflugerville,1104 Strickling Dr,78660,30.432673,-97.661697,295000.0,1,2020-10-13,10,2020,Single Family,6185.0,1768.0,4,2.0,1.98
2,2084491383,pflugerville,1408 Fort Dessau Rd,78660,30.409748,-97.639771,256125.0,1,2019-07-31,7,2019,Single Family,7840.0,1478.0,3,2.0,1.98
3,120901374,pflugerville,1025 Strickling Dr,78660,30.432112,-97.661659,240000.0,4,2018-08-08,8,2018,Single Family,6098.0,1678.0,3,2.0,1.98
4,60134862,pflugerville,15005 Donna Jane Loop,78660,30.437368,-97.656860,239900.0,3,2018-10-31,10,2018,Single Family,6708.0,2132.0,3,3.0,1.98
5,2080105342,pflugerville,14908 Spring Hill Ln,78660,30.437775,-97.656181,309045.0,2,2020-08-05,8,2020,Single Family,5161.0,1446.0,3,2.0,1.98
6,241932337,pflugerville,14009 Cantata Ln,78660,30.414684,-97.643394,315000.0,2,2020-06-11,6,2020,Single Family,12196.8,2432.0,4,3.0,1.98
7,60134835,pflugerville,15009 Drusillas Dr,78660,30.439404,-97.658035,219900.0,2,2018-10-10,10,2018,Single Family,5140.0,1422.0,3,3.0,1.98
8,60134833,pflugerville,1104 Traci Michelle Dr,78660,30.439096,-97.657959,225000.0,1,2019-09-19,9,2019,Single Family,3811.0,1870.0,3,2.0,1.98
9,83827468,del valle,6808 Plains Crest Dr,78617,30.160702,-97.638123,194800.0,1,2018-09-11,9,2018,Single Family,5183.0,1422.0,3,3.0,1.98


In [19]:
# Check length of rows
length1 = len(Austin_Housing_Market_df)
print(length1)

15171


In [20]:
# Check for null values in the dataset
Austin_Housing_Market_df.isnull().values.any()

False

In [21]:
# Check for duplicates of Zillow ID: Will return all duplicated rows. If nothing returned, there are no duplicates. 
Austin_Housing_Market_df[Austin_Housing_Market_df.duplicated(['zpid'], keep=False)]

,zpid,city,streetAddress,zipcode,latitude,longitude,latestPrice,numPriceChanges,latest_saledate,latest_salemonth,latest_saleyear,homeType,lotSizeSqFt,livingAreaSqFt,numOfBedrooms,numOfBathrooms,propertyTaxRate


In [22]:
# Check for duplicates of homes by street address: Will return all duplicated rows. 
# If nothing returned, there are no duplicates.
Austin_Housing_Market_df[Austin_Housing_Market_df.duplicated(['streetAddress'], keep=False)]

,zpid,city,streetAddress,zipcode,latitude,longitude,latestPrice,numPriceChanges,latest_saledate,latest_salemonth,latest_saleyear,homeType,lotSizeSqFt,livingAreaSqFt,numOfBedrooms,numOfBathrooms,propertyTaxRate
1462,2111911001,austin,2604 Arion Cir,78730,30.335613,-97.829010,160000.0,3,2018-08-11,8,2018,Single Family,17424.0,3600.0,4,4.0,1.98
1469,125904436,austin,2604 Arion Cir,78730,30.335608,-97.829002,1075000.0,1,2018-08-11,8,2018,Single Family,17424.0,3629.0,4,4.0,1.98
1914,250400973,austin,238 Lavaca Heights Dr,78737,30.183710,-97.988678,412500.0,1,2020-04-02,4,2020,Single Family,8276.0,1913.0,3,2.0,2.01
4201,2088737292,austin,238 Lavaca Heights Dr,78737,30.182819,-97.986626,399990.0,2,2020-04-02,4,2020,Single Family,8276.0,1913.0,3,2.0,2.01
4703,305425899,austin,4716 Amarra Dr,78735,30.268053,-97.883705,3175000.0,1,2020-08-10,8,2020,Single Family,87991.2,4853.0,4,6.0,1.98
5048,89548952,austin,11209 Lost Maples Trl,78748,30.155748,-97.832542,214900.0,1,2018-04-27,4,2018,Townhouse,5140.0,1236.0,2,3.0,1.98
5341,2088738491,austin,4716 Amarra Dr,78735,30.267832,-97.883476,3175000.0,1,2020-05-05,5,2020,Vacant Land,87120.0,4853.0,4,6.0,1.98
5627,2092180147,austin,10816 Mickelson Dr,78747,30.133886,-97.774887,312400.0,3,2019-08-09,8,2019,Single Family,8999.0,2690.0,5,3.0,1.98
5690,251030610,austin,10816 Mickelson Dr,78747,30.133755,-97.775002,330000.0,1,2019-08-09,8,2019,Single Family,8999.0,2690.0,5,3.0,1.98
6594,89547387,austin,2404 Independence Dr,78745,30.214905,-97.806824,275000.0,4,2020-06-05,6,2020,Single Family,9016.0,1781.0,4,2.0,1.98


In [23]:
# Count of rows with same street address
len(Austin_Housing_Market_df[Austin_Housing_Market_df.duplicated(['streetAddress'], keep=False)])

14

In [24]:
# Drop rows with same address
Austin_Housing_Market_df.drop_duplicates(subset ='streetAddress', keep=False, inplace = True)

In [31]:
# Use get_dummies to turn strings into usable data for machine learning model
Austin_Housing_Market_df = pd.get_dummies(Austin_Housing_Market_df, columns = ['city', 'homeType',])
Austin_Housing_Market_df.head()

,zpid,streetAddress,zipcode,latitude,longitude,latestPrice,numPriceChanges,latest_saledate,latest_salemonth,latest_saleyear,...,homeType_Apartment,homeType_Condo,homeType_Mobile / Manufactured,homeType_MultiFamily,homeType_Multiple Occupancy,homeType_Other,homeType_Residential,homeType_Single Family,homeType_Townhouse,homeType_Vacant Land
0,111373431,14424 Lake Victor Dr,78660,30.430632,-97.663078,305000.0,5,2019-09-02,9,2019,...,0,0,0,0,0,0,0,1,0,0
1,120900430,1104 Strickling Dr,78660,30.432673,-97.661697,295000.0,1,2020-10-13,10,2020,...,0,0,0,0,0,0,0,1,0,0
2,2084491383,1408 Fort Dessau Rd,78660,30.409748,-97.639771,256125.0,1,2019-07-31,7,2019,...,0,0,0,0,0,0,0,1,0,0
3,120901374,1025 Strickling Dr,78660,30.432112,-97.661659,240000.0,4,2018-08-08,8,2018,...,0,0,0,0,0,0,0,1,0,0
4,60134862,15005 Donna Jane Loop,78660,30.437368,-97.656860,239900.0,3,2018-10-31,10,2018,...,0,0,0,0,0,0,0,1,0,0


In [32]:
# Check length of rows
length2 = len(Austin_Housing_Market_df)
print(length2)

15157


In [33]:
# Check data types for each column
Austin_Housing_Market_df.dtypes

zpid                                int64
streetAddress                      object
zipcode                             int64
latitude                          float64
longitude                         float64
latestPrice                       float64
numPriceChanges                     int64
latest_saledate                    object
latest_salemonth                    int64
latest_saleyear                     int64
lotSizeSqFt                       float64
livingAreaSqFt                    float64
numOfBedrooms                       int64
numOfBathrooms                    float64
propertyTaxRate                   float64
city_austin                         uint8
city_del valle                      uint8
city_driftwood                      uint8
city_dripping springs               uint8
city_manchaca                       uint8
city_manor                          uint8
city_pflugerville                   uint8
city_road                           uint8
city_west lake hills              

In [35]:
# Export dataframe to csv
Austin_Housing_Market_df.to_csv('Resources/AHM_cleaned.csv', index = False)